In [2]:
import numpy as np
import cvxopt as cvx
import pylab
from scipy.integrate import odeint

a_matrix = np.matrix([[-1, 1, 0, 0],
                    [1, 0, -1, 0],
                    [0, 1, 0, 1],
                    [0, 0, 1, 0]])
d_matrix = np.matrix([[1, 0, 0, 0],
                    [0, 1, 0, 0],
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]])
b = np.matrix([[0, 1, -1, 0]])
c = np.matrix([[0], [0], [0], [0]])
x = [-1, -1, -1, -1]
H = np.matrix([[0, 1, 1, 0],
             [1, -1, 0, -1]])
g = np.matrix([[-72.7564], [-207.8494]])
t_0 = 0
t = 8

ModuleNotFoundError: No module named 'cvxopt.base'

In [2]:
def func(omega, tt, A, c):
    return np.dot(A, omega)


def func1(y, tt1, A, b):
    return np.dot(A, y) + b


def prepareQuad(B, B0, D, H, N, c, g):
    D_new = cvx.matrix(np.dot(np.dot(B.transpose(), D), B))
    A_new = cvx.matrix(np.dot(H, B))
    c_new = cvx.matrix(np.dot(B.transpose(), c + np.dot(D, B0)))
    b_new = cvx.matrix(g - np.dot(H, B0))
    G = np.zeros((2 * int(N), int(N)))
    for i in xrange(int(N)):
        G[i][i] = 1
    for i in xrange(int(N)):
        G[int(N) + i][i] = -1
    G = cvx.matrix(G)
    h = [1.0 for _ in xrange(2 * int(N))]
    h = cvx.matrix(h)
    return A_new, D_new, G, b_new, c_new, h


def optimalControl(D, A, b, c, x_zv, H, g, t0, t_zv):
    N = 30
    y = np.zeros(len(A))
    h = (t_zv - t0) / N
    tj = []

    for i in xrange(1, int(N) + 1):
        tj.append(t0 + i * h)

    tt = [t0, t_zv]
    B0 = odeint(func, x_zv, tt, args=(A, b))[1]
    tt1 = [t0, tj[0]]
    y1 = odeint(func1, y, tt1, args=(A, b))[1]
    temp = odeint(func1, y1, tj, args=(A, b))
    B = np.random.random((len(A), int(N)))

    for i in xrange(int(N)):
        B[:, i] = temp[int(N) - i - 1]

    A_new, D_new, G, b_new, c_new, h = prepareQuad(B, B0, D, H, N, c, g)

    u = cvx.solvers.qp(D_new, c_new, G, h, A_new, b_new)['x']

    for i in xrange(0, int(N)):
        tj[i] -= (t_zv - t0) / N

    pylab.plot(tj, u)
    pylab.show()

In [ ]:
optimalControl(d_matrix, a_matrix, b, c, x, H, g, t_0, t)

In [6]:
import cvxopt as cvx

ModuleNotFoundError: No module named 'cvxopt.base'